In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import csv

training_examples = []
training_files = [f for f in os.listdir('./raw/') if f.endswith('.txt')]
training_size = len(training_files)
training_files.sort()
print(training_size)
print(training_files[0])

12000
132539.txt


In [2]:
y_dict = {}
for file in ['Outcomes-a.txt', 'Outcomes-b.txt', 'Outcomes-c.txt']:
    with open(file, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i == 0:
                continue
            y_dict[row[0]] = int(row[-1])

In [3]:
static_name = ['Age', 'Gender', 'Height', 'ICUType']
dynamic_name = [
    'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN',
    'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT', 'HR', 'K', 'Lactate', 'Mg',
    'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 'PaO2', 'pH', 'Platelets', 'RespRate',
    'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC', 'Weight'
]
static_num = len(static_name)
dynamic_num = len(dynamic_name)
print(static_num, dynamic_num)

4 37


In [ ]:
static_dict = {static_name[i]: i for i in range(static_num)}
dynamic_dict = {dynamic_name[i]: i for i in range(dynamic_num)}
x = []
static = []
y = []
name = []
mask = []
static_mask = []

for file in training_files:
    x_one = []
    mask_one = []
    static_one = np.zeros((static_num))
    static_mask_one = np.ones((static_num))
    with open(os.path.join('raw', file), mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        x_row = np.zeros((dynamic_num))
        mask_row = np.zeros((dynamic_num))
        cur_time = 0
        for i, row in enumerate(reader):
            if i == 0:
                continue
            if row[1] == 'RecordID':
                continue
            time = int(row[0][:2])
            while time > cur_time:
                cur_time += 1
                x_one.append(np.divide(x_row, mask_row, out=np.zeros_like(x_row), where=mask_row!=0))
                mask_one.append(mask_row)
                x_row = np.zeros((dynamic_num))
                mask_row = np.zeros((dynamic_num))
            if row[1] in static_dict:
                if float(row[2]) == -1:
                    static_mask_one[static_dict[row[1]]] = 0
                static_one[static_dict[row[1]]] = float(row[2])
            elif row[1] in dynamic_dict:
                if float(row[2]) != -1:
                    x_row[dynamic_dict[row[1]]] += float(row[2])
                    mask_row[dynamic_dict[row[1]]] += 1
            else:
                print(file, i, row)
#                 raise Exception(row[1] + "不存在！")
    if len(x_one) >= 1:
        x.append(x_one)
        mask.append(mask_one)
        static.append(static_one)
        y.append(y_dict[file.split('.')[0]])
        name.append(int(file.split('.')[0]))
        static_mask.append(static_mask_one)
    else:
        print(int(file.split('.')[0]))
print(len(x), len(mask), len(static), len(y), len(name), len(static_mask))

In [5]:
static_masked = np.ma.masked_array(static, static_mask == 0)
static_mean = np.mean(static_masked, 0)
static_std = np.std(static_masked, 0)
print(static_mean)
print(static_std)

[64.54446112779446 0.5593093093093093 88.2329829829832 2.7789456122789455]
[17.167085289459482 0.49848210382242947 86.07397893514587
 1.0174571449222138]


In [6]:
x_flat = []
mask_flat = []
for i in range(len(x)):
    x_flat += x[i]
    mask_flat += mask[i]
x_masked = np.ma.masked_array(x_flat, np.array(mask_flat) == 0)
x_mean = np.mean(x_masked, 0)
x_std = np.std(x_masked, 0)
print(x_mean)
print(x_std)

[2.8905121746431606 119.88992256399226 359.16417145844224
 499.3405812251725 2.8602379720641435 27.168986262665626
 155.77648839556005 1.4718174102812507 59.548933967622546 0.54334321190119
 11.400131317707816 141.0752710430305 23.14264098352496 30.693924820034812
 86.67964960905361 4.13046925943177 2.9582480911146423 2.0226436037829645
 80.28002073734399 1.0 139.103615805169 58.42041704550531
 77.38323911303443 119.70474122706251 40.40811903927159 148.07255714038462
 7.422682125322088 190.06484593502736 19.589647712868015 96.68314821632046
 119.83976287909448 37.075962366405435 8.223261802575104 1.177369007263943
 116.80102491044838 13.11253574076491 83.31616226269492]
[0.6515312461096975 174.7038210819652 1127.2163081328217
 1629.1306842987667 5.7808879722598485 22.579892422277968 44.4468155994555
 1.548282599489897 12.853351581895854 0.1885805601618637
 3.9980440115359355 65.29745894882797 4.721950545729931 5.00915889640266
 17.787786845117104 0.682788998245779 2.557057698421397 0.5

In [7]:
print(x_mean[19], x_std[19])
x_mean[19] = 0
x_std[19] = 1

1.0 0.0


In [8]:
x_normalize = []
static_normalize = []
mask_normalize = []
for i in range(len(x)):
    # static_normalize.append(np.where(static_mask[i] > 0, (static[i] - static_mean) / static_std, 0).tolist())
    static_normalize.append(np.where(static_mask[i] > 0, static[i], 0).tolist())
    x_one = []
    mask_one = []
    for j in range(len(x[i])):
        x_one.append(np.where(mask[i][j] > 0, x[i][j], 0).tolist())
        # x_one.append(np.where(mask[i][j] > 0, (x[i][j] - x_mean) / x_std, 0).tolist())
        mask_one.append(np.where(mask[i][j] > 0, 1, 0).tolist())
    x_normalize.append(x_one)
    mask_normalize.append(mask_one)

In [ ]:
index_array = list(range(len(x)))
# np.random.shuffle(index_array)
print(index_array)

In [10]:
x_random = []
y_random = []
static_random = []
mask_random = []
name_random = []
for i in range(len(index_array)):
    x_random.append(x_normalize[index_array[i]])
    y_random.append(y[index_array[i]])
    static_random.append(static_normalize[index_array[i]])
    mask_random.append(mask_normalize[index_array[i]])
    name_random.append(name[index_array[i]])

In [11]:
pickle.dump((x_random, y_random, static_random, mask_random, name_random), open('data.pkl', 'wb'))
stat_dict = {'x_mean': x_mean, 'x_std': x_std, 'static_mean': static_mean, 'static_std': static_std}
pickle.dump(stat_dict, open('stat.pkl', 'wb'))

In [12]:
cnt = 0
cnt1 = 0
for i in range(len(mask)):
    for j in range(len(mask[i])):
        cnt += sum(mask[i][j])
        cnt1 += len(mask[i][j])
print('Observed Rate:', cnt / cnt1)

Observed Rate: 0.24722348952128317
